# Async IO

Asynchronous I/O is a form of input/output processing that permits other processing to continue before the transmission has finished. It is a single-threaded, single-process design. It is a style of concurrent programming, but it is not parallelism. 

In [1]:
import asyncio
import time

In [2]:
async def count():
    print("One")
    
    # task tells the outer event loop that it's going to sleep. Let other work continue.
    await asyncio.sleep(1)  
    # This is a stand-in for any time-intensive, but non-blocking, function call.
    
    print("Two")
    
async def main():
    await asyncio.gather(count(), count(), count())


If you ran this in a script, you'd run
```
asyncio.run(main())
```
which is the "normal" way to run `asyncio`. There is some magic going on to get this to work in ipython, because there's already a running event loop when you run ipython.

In [3]:
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed: .2f}s")

One
One
One
Two
Two
Two
Executed in  1.01s


Contrast this to synchronous code:

In [4]:
def count():
    print("One")
    time.sleep(1)
    print("Two")

def main():
    for _ in range(3):
        count()

s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed: .2f}s")

One
Two
One
Two
One
Two
Executed in  3.05s


## Concurrently make random numbers

In [5]:
import random

In [6]:
# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

Keep making random numbers until the number is above some threshold (with sleep inbetween)

In [7]:
async def makerandom(idx: int, threshold: int = 6):
    async_color = c[idx + 1]
    print(async_color+f"Initiated makerandom({idx})")
    i = random.randint(0, 10)
    while i <= threshold:
        print(async_color+f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)  # mimics an IO-bound process with an uncertain wait time
        i = random.randint(0, 10)
    print(async_color+f"--> Finished: makerandom({idx}) == {i}"+c[0])
    return i

In [8]:
async def main():
    # Concurrently run makerandom() across 3 different inputs
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

In [9]:
random.seed(444)

In [10]:
r1, r2, r3 = await main()
print()
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0)
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1)
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2)
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 8 too low; retrying.
--> Finished: makerandom(1) == 10
makerandom(0) == 7 too low; retrying.
makerandom(0) == 8 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(0) == 6 too low; retrying.
--> Finished: makerandom(2) == 9
makerandom(0) == 3 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 7 too low; retrying.
--> Finished: makerandom(0) == 10

r1: 10, r2: 10, r3: 9


## Async IO vs threading

- Threading scales less well than async IO because threads are a system resource with finite availability. Creating thousands of threads will fail on many machines, whereas creating thousands of async IO tasks is completely feasible.
- Async IO shines for IO bound tasks which would be otherwise dominated by blocking IO-bound wait-time, such as
    - Network IO
    - Read/write operations with a "fire-and-forget" style, without worrying about a lock for whatever you're reading/writing to
- The main disadvantage of async IO is that `await` only supports a specific set of objects that define a specific set of methods. 


## Mixing multiprocessing and async IO

This is a tricky business because a child process will inherit the event loop of the parent. Some plumbing is required to get async IO to play nice with multiprocessing. The `aiomultiprocess` library handles this for you, see `scripts/asyncio_multiprocessing.py` for an example. (I haven't figured out how to get this to run in a notebook yet)